In [1]:
from mygene import MyGeneInfo
import pandas as pd

# -----------------------------
# 配置文件路径
GENELIST_FILE = "/projectnb/perissilab/Xinyu/GPS2_CHIPseq/CTCF_3T3L1/results/annotation/GPS2_CTCF_common_genes.txt"
BED_FILE = "/projectnb/perissilab/Xinyu/GPS2_CHIPseq/mm39_ensembl.bed"
OUTPUT_FILE = "/projectnb/perissilab/Xinyu/GPS2_CHIPseq/CTCF_3T3L1/results/annotation/GPS2_CTCF_common_genes_filtered.bed"
# -----------------------------

# 1. 读取gene list
genes = pd.read_csv(GENELIST_FILE, header=None)[0].tolist()

# 2. 用mygene转换为Ensembl ID
mg = MyGeneInfo()
results = mg.querymany(genes, scopes='symbol', fields='ensembl.gene', species='mouse')

# 整理结果
df = pd.DataFrame(results)
df = df[df['notfound'] != True][['query','ensembl']]

def extract_ensembl(x):
    if isinstance(x, dict):
        return x.get('gene')
    elif isinstance(x, list):
        return x[0].get('gene')
    return None

df['ensembl_id'] = df['ensembl'].apply(extract_ensembl)
ensembl_ids = set(df['ensembl_id'].dropna())

# 3. 读取BED文件，过滤
bed_cols = ['chr','start','end','gene_id','score','strand','thickStart','thickEnd','itemRgb','blockCount','blockSizes','blockStarts']
bed = pd.read_csv(BED_FILE, sep='\t', header=None, names=bed_cols)

filtered_bed = bed[bed['gene_id'].isin(ensembl_ids)]

# 4. 保存结果
filtered_bed.to_csv(OUTPUT_FILE, sep='\t', header=False, index=False)

print(f"✅ Done! Filtered BED written to: {OUTPUT_FILE}")


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
153 input query terms found dup hits:	[('Gm26901', 2), ('Gm33222', 2), ('Gm6644', 2), ('Gm29083', 2), ('2900060B14Rik', 2), ('2610027F03Ri
14 input query terms found no hit:	['Bves', 'Fut11', 'LOC118567337', '2310039H08Rik', 'LOC102631992', 'Gm50217', 'Tmem88b', 'Sagsin1', 


✅ Done! Filtered BED written to: /projectnb/perissilab/Xinyu/GPS2_CHIPseq/CTCF_3T3L1/results/annotation/GPS2_CTCF_common_genes_filtered.bed
